Agent Evaluation

This notebook demonstrates different agent evaluation approaches by creating test scenarios and evaluation criteria.

## What we'll learn:

- Metrics
- Test Case
- Evaluator
- Approaches
    - black box, 
    - single step, 
    - trajectory
- Evaluation Report

## Setup
First, let's import the necessary libraries:

In [1]:
from pathlib import Path
import sys

nb_dir = Path(__file__).parent if "__file__" in globals() else Path.cwd()
parent = nb_dir.parent  # points to 03-building-agents
if str(parent) not in sys.path:
    sys.path.insert(0, str(parent))

In [2]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
import time
from typing import List, Dict
from dotenv import load_dotenv

from lib.agents import Agent, AgentState
from lib.state_machine import Run
from lib.messages import AIMessage
from lib.tooling import tool
from lib.evaluation import TestCase, AgentEvaluator, EvaluationResult

In [4]:
load_dotenv()

False

## Your Custom Tools

Design and implement tools that will showcase different aspects of agent behavior. Consider creating tools that:

- Require multi-step reasoning
- Handle different data types and formats
- Interact with external services or APIs
- Process complex inputs
- Have potential failure modes to test error handling


In [5]:
from typing import TypedDict, List, Literal, Optional

class ToolStatus(TypedDict, total=False):
    status: Literal["ok", "no_context", "error"]
    error: Optional[str]

class GameItem(TypedDict):
    Game: str
    Platform: str
    Score: int

class GetGamesResult(ToolStatus, total=False):
    games: List[GameItem]

@tool
def get_games(num_games: int = 1, top: bool = True) -> GetGamesResult:
    """
    Return top or bottom N games by score.

    Inputs:
      num_games (int): number of games to return (default 1)
      top (bool): True for top, False for bottom

    Returns schema:
      {
        "type": "object",
        "properties": {
          "status": {"type":"string","enum":["ok","no_context","error"]},
          "error": {"type":"string"},
          "games": {
            "type":"array",
            "items":{"type":"object","properties":{
              "Game":{"type":"string"},
              "Platform":{"type":"string"},
              "Score":{"type":"integer"}
            }}
          }
        }
      }
    """
    data: List[GameItem] = [
        {"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98},
        {"Game": "Super Mario Odyssey", "Platform": "Switch", "Score": 97},
        {"Game": "Metroid Prime", "Platform": "GameCube", "Score": 97},
        {"Game": "Super Smash Bros. Brawl", "Platform": "Wii", "Score": 93},
        {"Game": "Mario Kart 8 Deluxe", "Platform": "Switch", "Score": 92},
        {"Game": "Fire Emblem: Awakening", "Platform": "3DS", "Score": 92},
        {"Game": "Donkey Kong Country Returns", "Platform": "Wii", "Score": 87},
        {"Game": "Luigi's Mansion 3", "Platform": "Switch", "Score": 86},
        {"Game": "Pikmin 3", "Platform": "Wii U", "Score": 85},
        {"Game": "Animal Crossing: New Leaf", "Platform": "3DS", "Score": 88},
    ]
    if num_games <= 0:
        return {"status": "no_context", "games": []}
    sorted_games = sorted(data, key=lambda x: x["Score"], reverse=top)
    return {"status": "ok", "games": sorted_games[:num_games]}

## Develop your Agent

In [6]:
agent = Agent(
    model_name="gpt-4o-mini",
    tools=[get_games],
    instructions="You can bring insights about a game dataset based on users questions",
)

## Design Your Test Cases

In [7]:
evaluator = AgentEvaluator()

In [8]:
test_cases = [
    TestCase(
        id="game_query_1",
        description="Find the best game in the dataset",
        user_query="What's the best game in the dataset?",
        expected_tools=["get_games"],
        reference_answer="The Legend of Zelda: Breath of the Wild with score 98",
        max_steps=4
    ),
]

## Run Your Evaluation Experiments

In [9]:
for test_case in test_cases:
    print(f"\n=== Evaluating Test Case: {test_case.id} ===")
    print(f"Description: {test_case.description}")
    print(f"Query: {test_case.user_query}")
    
    # Run the agent
    start_time = time.time()
    print("\nWorkflow:")
    agent.memory.reset()
    run_object:Run = agent.invoke(test_case.user_query)
    execution_time = time.time() - start_time
    
    # Get final state and response
    final_state:AgentState = run_object.get_final_state()
    if final_state and final_state.get("messages"):
        # Find the last AI message as the final response
        final_response = ""
        for msg in reversed(final_state["messages"]):
            if isinstance(msg, AIMessage) and msg.content:
                final_response = msg.content
                break
        
        total_tokens = final_state.get("total_tokens", 0)
        
        # Evaluate using all three methods
        print("\n--- Black Box (Final Response) Evaluation ---")
        black_box_eval:EvaluationResult = evaluator.evaluate_final_response(
            test_case, final_response, execution_time, total_tokens
        )
        print(f"Overall Score: {black_box_eval.overall_score:.2f}")
        print(f"Task Completed: {black_box_eval.task_completion.task_completed}")
        print(f"Feedback: {black_box_eval.feedback}")
        
        print("\n--- Single Step Evaluation ---")
        step_eval:EvaluationResult = evaluator.evaluate_single_step(
            final_state["messages"], test_case.expected_tools
        )
        print(f"Overall Score: {step_eval.overall_score:.2f}")
        print(f"Correct Tool Selected: {step_eval.tool_interaction.correct_tool_selected}")
        print(f"Feedback: {step_eval.feedback}")
        
        print("\n--- Trajectory Evaluation ---")
        traj_eval:EvaluationResult = evaluator.evaluate_trajectory(test_case, run_object)
        print(f"Overall Score: {traj_eval.overall_score:.2f}")
        print(f"Steps Taken: {traj_eval.task_completion.steps_taken}")
        print(f"Total Tokens: {traj_eval.system_metrics.total_tokens}")
        print(f"Execution Time: {traj_eval.system_metrics.execution_time:.2f}s")
        print(f"Estimated Cost: ${traj_eval.system_metrics.cost_estimate:.6f}")
        print(f"Feedback: {traj_eval.feedback}")
        
    else:
        print("ERROR: No final state or messages found")


=== Evaluating Test Case: game_query_1 ===
Description: Find the best game in the dataset
Query: What's the best game in the dataset?

Workflow:
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

--- Black Box (Final Response) Evaluation ---
Overall Score: 1.00
Task Completed: True
Feedback: The agent response successfully identifies the best game in the dataset, providing the name, platform, and score, which fully answers the user's query. The format is appropriate as it clearly presents the information. Additionally, the response follows the implicit instruction to provide a clear and concise answer.

--- Single Step Evaluation ---
Overall Score: 1.00
Correct Tool Selected: True
Feedback: Selected tools: ['get_games'], Expected: ['get_games']

--- Trajectory Evaluation ---
Overa

## Reflection and Iteration

Based on your evaluation results:

1. **Identify Patterns**: What trends do you see in your agent's performance?
2. **Spot Weaknesses**: Where does your agent struggle most?
3. **Recognize Strengths**: What does your agent do particularly well?
4. **Iterate**: Modify your tools, test cases, or agent configuration and re-evaluate


# Patterns
  
  - Consistent tool selection when the query maps clearly to one tool.
  - Short trajectories (≈4 steps) and reasonable token use (hundreds per run).
  - Final responses concise and on-target for well-scoped prompts.

# Weaknesses

  - Limited coverage: few test cases means blind spots (ambiguous queries, missing params, multi-step reasoning).
  - Fallback behavior only triggers when tools return “no_context”; needs broader tests (conflicting tools, empty results, errors).
  - Little stress on token budget (many tools, long prompts, long chats) in evaluations.

# Strengths

  - Unified tool return envelope (status/no_context/error) works; agent can reason on structured outputs.
  - Auto-included tool docs and return schemas reduce prompt handcrafting.
  - Token measurement and budget pruning integrated; observable total tokens and costs.

# Iterate (next actions)

## Expand test coverage:

  - Ambiguous queries (should ask clarifying Qs before tool use).
  - Missing parameters (ensure “ask before call” behavior).
  - Multi-tool flows (e.g., search → compare → answer).
  - Error paths (tool errors/timeouts/empty results).

## Strengthen fallback/routing:

  - Add tests where first tool returns “ok” but low-signal answer; require trying another tool.
  - Add a simple “confidence” heuristic (e.g., answer length/keywords) to gate fallback.

## Tighten token discipline:

  - Add tests with many tools/long history; verify pruning and summaries kick in.
  - Log per-step tokens and cumulative_tokens in trajectory metrics.

## Dial reliability:

  - Lower temperature for evaluations (e.g., 0.1–0.2) for consistency.
  - Add one-shot examples in instructions for tricky patterns (only if not fine-tuning).

## Data realism:

  - Add noisy/edge “game” entries and verify sorting/formatting.
  - Add adversarial prompts to validate tool doc adherence and injection resistance.

## Optional enhancements

  - Include tool latency and success/failure rates in evaluation output.
  - Add a “return format contract” test (JSON shape validation against return schema).